<a href="https://colab.research.google.com/github/ggnanaraj/Story/blob/main/Stock0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nselib

In [3]:
import nselib
#data = nselib.trading_holiday_calendar()
#print(data)

In [ ]:
from nselib import capital_market

In [ ]:
data=capital_market.bhav_copy_with_delivery("01-03-2024")
print(data.columns)

Index(['SYMBOL', 'SERIES', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE',
       'CLOSE_PRICE', 'PREV_CLOSE', 'TTL_TRD_QNTY', 'TURNOVER_LACS',
       'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER', 'DATE1'],
      dtype='object')


In [ ]:
#data1=data('SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER')
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)

##data1.sort_values(by=['GAP_UP'],ascending=False)
#data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
#data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
#data_VOL=data_5PC_GAP_UP[data_5PC_GAP_UP['TTL_TRD_QNTY'] > 50000] #Quantity greater than 50000
#data_VOL1=data_VOL[data_VOL['DELIV_PER'] != "-"]
##data_VOL1[data_VOL1['DELIV_PER'] > 5]
#data_VOL1.DELIV_PER=data_VOL1.DELIV_PER.astype(float)
#data_VOL1=data_VOL1[data_VOL1['DELIV_PER'] > 20]  #
#data_VOL1=data_VOL1[data_VOL1['DELIV_PER'] < 90]
#data_VOL1['GAP_down']=(((data_VOL1['HIGH_PRICE']-data_VOL1['LOW_PRICE'])/data_VOL1['HIGH_PRICE'])*100).round(2)
#data_VOL1





In [30]:
import nselib
from nselib import capital_market
data=capital_market.bhav_copy_with_delivery("07-03-2024")
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)
data1['TTL_TRD_QNTY']=0
data1['DELIV_PER']=0

#data1=data1[data1['DELIV_PER'] != "-"]
data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
data_PC=data_5PC_GAP_UP[data_5PC_GAP_UP['PREV_CLOSE'] > 10] # only share value above Rs10
data_PC=data_PC.sort_values(by=['SYMBOL'])
list_SYM=data_PC['SYMBOL'].to_list()
list_SYM.sort()
#---------------------------------
dataO=capital_market.bhav_copy_with_delivery("26-02-2024")
data1O=dataO.loc[1:,['SYMBOL','TTL_TRD_QNTY','DELIV_PER']]
data1O=data1O[data1O['DELIV_PER'] != "-"]
data2O=data1O[data1O['SYMBOL'].isin(list_SYM)]# list_SYM or data_PC or list_PC
data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
data2O=data2O.sort_values(by=['SYMBOL'])
listO_SYM=data2O['SYMBOL'].to_list()
listO_SYM.sort()
#---------------------------
list_comn=[symbol for symbol in list_SYM if symbol in listO_SYM]
list_diff=[symbol for symbol in list_SYM if symbol not in listO_SYM]
list_indexDel=[]
data_PC1= data_PC
for i in list_diff:
  list_1=data_PC.index[data_PC['SYMBOL'] == i ].to_list()
  data_PC1.drop(list_1, inplace=True) # drops row which is not common
  list_indexDel.extend(list_1)
for j,k in enumerate(list_comn):
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']= int(PC1)
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'DELIV_PER']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'DELIV_PER']= float(PC1)

#data2 = capital_market.price_volume_and_deliverable_position_data(symbol='SBIN', from_date='29-02-2024', to_date='01-03-2024')
#data3=capital_market.equity_list( )
data_VOL=data_PC1[data_PC1['TTL_TRD_QNTY'] > 50000] #Quantity greater than 50000
data_VOL=data_VOL[data_VOL['DELIV_PER'] != "-"]
#data_VOL=data_VOL[data_VOL['DELIV_PER'] > 20]  #
#data_VOL=data_VOL[data_VOL['DELIV_PER'] < 90]
data_VOL['GAP_down']=(((data_VOL['HIGH_PRICE']-data_VOL['LOW_PRICE'])/data_VOL['HIGH_PRICE'])*100).round(2)
data_VOL['INT_Hike']=(((data_VOL['HIGH_PRICE']-data_VOL['OPEN_PRICE'])/data_VOL['OPEN_PRICE'])*100).round(2)
data_VOL


<ipython-input-30-2ff5f6ff842c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate


,SYMBOL,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRICE,PREV_CLOSE,GAP_UP,TTL_TRD_QNTY,DELIV_PER,GAP_down,INT_Hike
522,DATAPATTNS,2797.70,2797.70,2665.00,2698.10,2634.40,6.20,2464376,18.73,4.74,0.00
1208,JWL,398.00,398.00,368.00,369.90,362.05,9.93,1152870,38.62,7.54,0.00
1536,MOS,168.80,168.80,157.00,160.00,159.00,6.16,414400,38.61,6.99,0.00
1823,PRECISION,52.85,52.85,51.10,51.60,50.15,5.38,56000,89.29,3.31,0.00
2329,SYRMA,569.95,569.95,522.50,525.55,519.80,9.65,835423,48.78,8.33,0.00
2521,VENUSREM,345.80,345.80,330.55,334.20,327.60,5.56,79949,56.54,4.41,0.00
2596,WSI,146.95,167.85,141.05,150.50,139.90,5.04,187078,63.61,15.97,14.22


In [34]:
#len(data_PC1)
#len(data2O)
list_comn


['DATAPATTNS',
 'IBULHSGFIN',
 'IBULHSGFIN',
 'INDIFRA',
 'INFINIUM',
 'INSPIRE',
 'IPSL',
 'JWL',
 'KORE',
 'LEXUS',
 'LRRPL',
 'MOS',
 'PRECISION',
 'QUICKTOUCH',
 'ROML',
 'SAHAJ',
 'SHRITECH',
 'SHYAMTEL',
 'SYRMA',
 'TRANSTEEL',
 'VENUSREM',
 'WSI']

In [10]:
"""PC1=data2O.loc[data2O['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']
data_PC1.loc[data_PC1['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']= int(PC1)
PC1=data2O.loc[data2O['SYMBOL']==listO_SYM[0],'DELIV_PER']
data_PC1.loc[data_PC1['SYMBOL']==listO_SYM[0],'DELIV_PER']= float(PC1)
#data2O.loc[data2O['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']
#data_PC1['TTL_TRD_QNTY'].astype('Int64')
#PC1"""
data_PC1



,SYMBOL,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRICE,PREV_CLOSE,GAP_UP,TTL_TRD_QNTY,DELIV_PER
63,AARTISURF,348.00,348.00,303.00,325.50,330.00,5.45,17482,39.59
155,AMBICAAGAR,34.50,34.50,30.40,30.80,31.80,8.49,129190,47.66
447,CGPOWER,480.50,494.95,461.00,465.00,443.25,8.40,7113044,53.65
496,COOLCAPS,459.00,460.00,445.00,445.00,431.00,6.50,2000,100.00
580,DHTL,223.00,223.00,207.00,209.05,212.35,5.02,41600,69.23
605,DOLLEX,49.00,49.25,45.95,46.40,46.20,6.06,36000,55.56
955,HOMESFY,468.95,468.95,455.00,455.00,437.30,7.24,600,100.00
1272,KLL,101.55,107.80,97.05,106.40,93.90,8.15,643200,84.58
1330,LANDMARK,777.95,789.95,748.00,758.35,719.65,8.10,119030,51.28
1340,LEMERITE,59.85,59.85,56.15,58.05,56.50,5.93,28800,50.00
